In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report, recall_score, precision_score, confusion_matrix


In [2]:
airogs = pd.read_pickle('airogs_df')
refuge = pd.read_pickle('refuge_df')
gamma = pd.read_pickle('gamma_df')
all_datasets = {'AIROGS': airogs, 'REFUGE': refuge, 'GAMMA': gamma}


In [3]:
class Metric:
    threshold = 0.5
    def metrics(self, ):
        return {'AUROC':self.auroc, 'Sens@95Spec': self.sens95spec, 'Recall': self.recall, 'Precision': self.precision, 'Specificity': self.specificity, 'NPV': self.npv}
    def sens95spec(self, y, probas):
        fpr, tpr, _ = roc_curve(y, probas, drop_intermediate=True)
        spec95, spec95idx = find_nearest(fpr, 0.05)
        sens95 = tpr[spec95idx]
        return sens95
    def auroc(self, y, probas):
        auroc = roc_auc_score(y, probas)
        return auroc
    def specificity(self, y, probas):
        confMat = confusion_matrix(y, probas>self.threshold)
        tn, fp, fn, tp = confMat.ravel()
        specificity = tn / (tn+fp)
        return specificity
    def npv(self, y, probas):
        confMat = confusion_matrix(y, probas>self.threshold)
        tn, fp, fn, tp = confMat.ravel()
        pn = fn + tn
        return tn/pn
    def precision(self, y, probas):
        return precision_score(y, probas>self.threshold)
    def recall(self, y, probas):
        return recall_score(y, probas>self.threshold)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    arr_value = array[idx]
    return  arr_value, idx



In [4]:
def boostrap_CI(y, probas, n_samples, metric):
    scores = []
    ref = metric(y, probas)
    N = len(y)
    for i in range(n_samples):
        indices = np.random.randint(0, N, N)
        y_sampled = y[indices]
        probas_sampled = probas[indices]
        score = metric(y_sampled, probas_sampled)
        scores.append(score)
    # calculate 95% confidence intervals (100 - alpha)
    alpha = 5.0
    # calculate lower percentile (e.g. 2.5)
    lower_p = alpha / 2.0
    upper_p = (100 - alpha) + (alpha / 2.0)
    median = np.median(scores)
    lower = max(0.0, np.percentile(scores, lower_p))
    upper = min(1.0, np.percentile(scores, upper_p))
    return ref, median, lower, upper

In [5]:
metric = Metric()
all_scores = {}
thresholds = [0.1, 0.3, 0.5]
for mname, m in metric.metrics().items():
    for dname, data in all_datasets.items():
        for t in thresholds:
            metric.threshold = t
            ref, median, lower, upper = boostrap_CI(data['Label'], data['Positive pred'], 2000, m)

            col = f"{mname} (t={t:.1f})"
            col_low_CI = f"{col} 95-CI lower"
            col_up_CI = f"{col} 95-CI upper"
            if col not in all_scores:
                all_scores[col] = []
                all_scores[col_low_CI] = []
                all_scores[col_up_CI] = []
            all_scores[col].append(ref)
            all_scores[col_low_CI].append(lower)
            all_scores[col_up_CI].append(upper)
            

In [6]:
all_scores['datasets'] = list(all_datasets.keys())


In [7]:
df = pd.DataFrame.from_dict(all_scores)
df = df.set_index('datasets')
df.to_excel('all_scores.xlsx')

In [27]:
from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.io import curdoc

curdoc().clear()


In [28]:
datasets = list(df.index)
metrics = ['Specificity', 'Recall', 'Precision', 'NPV']
all_plots = []
output_notebook()

for mname in metrics:
    x = [(d, f"{t:.1f}") for d in datasets for t in thresholds]
    
    xs = [_.tolist() for _ in np.split(np.asarray(x), 3)]
    
    y = [df[f"{mname} (t={t:.1f})"][d] for d in datasets for t in thresholds]
    y_ci_lower = [df[f"{mname} (t={t:.1f}) 95-CI lower"][d] for d in datasets for t in thresholds]
    y_ci_upper = [df[f"{mname} (t={t:.1f}) 95-CI upper"][d] for d in datasets for t in thresholds]
    ys =  [_.tolist() for _ in np.split(np.asarray(y), 3)]
    
    ys_ci_lower = [_.tolist() for _ in np.split(np.asarray(y_ci_lower), 3)]
    ys_ci_upper = [_.tolist() for _ in np.split(np.asarray(y_ci_upper), 3)]
    source = ColumnDataSource(data=dict(x=x, y=y, y_ci_low=y_ci_lower, y_ci_up=y_ci_upper))
    p = figure(x_range=FactorRange(*x), height=250, title=f"{mname}", y_range=(0.5, 1.1),
           toolbar_location=None, tools="")
    
    colors = ["#179299", "#d20f39", '#40a02b']
    for i in range(3):
        p.varea(x=xs[i], y1=ys_ci_lower[i], y2=ys_ci_upper[i], alpha=0.15, fill_color=colors[i])
        p.scatter(x=xs[i], y=ys[i], color=colors[i])
    p.multi_line(xs=xs, ys=ys, width=1.2, color=colors, line_width=2)
    all_plots.append(p)
all_plots = [_.tolist() for _ in np.split(np.asarray(all_plots), 2)]
grid = gridplot(all_plots, merge_tools=True,)
grid.toolbar.logo = None

Loading BokehJS ...

In [29]:
show(grid)

In [30]:
export_png(grid, filename='all_metrics.png')

'/home/clement/Documents/postdoc/Glaucoma_referability/notebooks/all_metrics.png'